### Постановка задачи

https://youtu.be/X-iSQQgOd1A?si=gHvE1jqQz4nYu6CN


Идея муравьиного алгоритма – моделирование поведения муравьёв, связанного с их способностью быстро
находить кратчайший путь от муравейника к источнику пищи и адаптироваться к изменяющимся условиям,
находя новый кратчайший путь. При своём движении муравей метит путь феромоном, и эта информация
используется другими муравьями для выбора пути. Это элементарное правило поведения и определяет
способность муравьёв находить новый путь, если старый оказывается недоступным.
Рассмотрим случай, когда на оптимальном доселе пути возникает преграда. В этом
случае необходимо определение нового оптимального пути. Дойдя до преграды, муравьи с равной
вероятностью будут обходить её справа и слева. То же самое будет происходить и на обратной стороне
преграды. Однако, те муравьи, которые случайно выберут кратчайший путь, будут быстрее его проходить, и за
несколько передвижений он будет более обогащён феромоном. Поскольку движение муравьёв определяется
концентрацией феромона, то следующие будут предпочитать именно этот путь, продолжая обогащать его
феромоном до тех пор, пока этот путь по какой-либо причине не станет недоступен. 

Очевидная положительная обратная связь быстро приведёт к тому, что кратчайший путь станет единственным
маршрутом движения большинства муравьёв. Моделирование испарения феромона – отрицательной обратной
связи – гарантирует нам, что найденное локально оптимальное решение не будет единственным – муравьи
будут искать и другие пути. Если мы моделируем процесс такого поведения на некотором графе, рёбра
которого представляют собой возможные пути перемещения муравьёв, в течение определённого времени, то
наиболее обогащённый феромоном путь по рёбрам этого графа и будет являться решением задачи,
полученным с помощью муравьиного алгоритма.

Любой муравьиный алгоритм, независимо от модификаций, представим в следующем виде
- Пока (условия выхода не выполнены)
1. Создаём муравьёв
2. Ищем решения
3. Обновляем феромон
4. Дополнительные действия {опционально} 



Класс пути

In [1]:
from math import inf
from random import random, shuffle, randint

class Path:
    def __init__(self, points, length) -> None:
        self.points = points
        self.length = length

Класс базовой геометрии задачи, обеспечивающий вычисление пути по графу

In [2]:
class Base:
    @staticmethod
    def distance(a: 'tuple[int]', b: 'tuple[int]') -> float:
        """вычисление растояния между двумя точками"""
        return ((a[0] - b[0]) ** 2 + (a[1] - b[1]) ** 2) ** 0.5

    @staticmethod
    def path_distance(dist_matrix: 'list[list[float]]', indices: 'list[int]') -> float:
        """вычисление длины пути"""
        dist = 0
        for i, j in zip(indices[:-1], indices[1:]):
            dist += dist_matrix[i][j]
        return dist

    @staticmethod
    def distance_matrix(points: 'list[tuple[int]]') -> 'list[list[float]]':
        """вычисляет матрицу растояний"""
        return [[Base.distance(a, b) for b in points] for a in points]
  

Класс основного алгоритма

1. Создаём муравьёв

- Стартовая точка, куда помещается муравей, зависит от ограничений, накладываемых условиями задачи. Потому что для каждой задачи способ размещение муравьёв является определяющим. Либо
все они помещаются в одну точку, либо в разные с повторениями, либо без повторений. 
- На этом же этапе задаётся начальный уровень феромона. Он инициализируется небольшим положительным числом для того, чтобы на начальном шаге вероятности перехода в следующую
вершину не были нулевыми.
2. Ищем решения
- Вероятность перехода из вершины $i$ в вершину $j$ определяется по следующей формуле:
$$P_{ij}(t)=\frac{\tau_{ij}(t)^a \left( \frac{1}{d_{ij}}\right)^b}{\sum_{k}{\tau_{ik}(t)^a \left( \frac{1}{d_{ik}}\right)^b}}$$

Где

- $\tau_{ij}(t)$ – уровень феромона,
- $d_{ij}$ – эвристическое расстояние,
- $a,b$ – константные параметры.

При $a$ = 0 выбор ближайшего города наиболее вероятен, то есть алгоритм становится жадным.
При $b$ = 0 выбор происходит только на основании феромона, что приводит к субоптимальным решениям.

Поэтому необходим компромисс между этими величинами, который находится экспериментально.

3. Обновляем феромон

- Уровень феромона обновляется в соответствии с формулой
$$\tau_{ij}(t + 1) = (1-\rho)\tau_{ij}(t) + \sum_{k}{\frac{Q}{L_k}}$$

Где 
- $ρ$ – интенсивность испарения,
- $L_k$ – цена текущего решения для k-ого муравья,
- $Q$ – параметр, имеющий значение порядка цены оптимального решения
- $\frac{Q}{L_k}$ - феромон, откладываемый $k$-ым муравьём, использующим ребро ($i$, $j$).

4. Дополнительные действия
- Обычно здесь используется алгоритм локального поиска, однако он может также появиться и после
поиска всех решений.

In [ ]:
class AntColonyOptimization(Base):
    """
    "Муравьиный" алгоритм построен на моделировании поведения муравья, который ищет кратчайший путь между муравейником и едой.\n
    -----
    `num_ants: int` количество муравьев\n   
    `num_histories: int` количество итераций\n
    `a: float` параметр модели, фактор извлечения информации, отражает важность накопления феромона для выбора пути муравьев.\n
    `b: float` параметр модели, ожидаемый эвристический фактор, отражает важность эвристической информации для выбора пути муравьев.\n
    `p: float` коэффициент испарения феромонов, предотвращает бесконечное накопление феромонов, обычно в диапазоне [0, 1]\n
    `q: float` интенсивность феромонов, представляет собой общее количество феромонов, в определенной степени влияет на скорость сходимости алгоритма\n    
    """

    def __init__(self, num_ants: int, num_histories: int, a: float, b: float, p: float, q: float) -> None:
        self.num_ants = num_ants
        self.num_histories = num_histories
        self.temp_res = []
        self.a = a
        self.b = b
        self.p = p
        self.q = q

    @staticmethod
    def select_index(selection: 'list[int]') -> int:
        """Выбирает случайный индекс следующей точки."""
        s = sum(selection)
        if s == 0:
            return len(selection) - 1
        alpha = random()
        p = 0
        for i in range(len(selection)):
            p += selection[i] / s
            if p >= alpha:
                return i

    def create_index(self, dist_matrix: 'list[list[float]]', pheromone_matrix: 'list[list[float]]') -> 'list[int]':
        """Создает новый порядок индексов точек на основе расстояния и феромона."""
        l = len(dist_matrix)
        unvisited_indices = list(range(l))
        shuffle(unvisited_indices)
        visited_index = [unvisited_indices.pop()]
        for _ in range(l - 1):
            i = visited_index[-1]
            selection = []
            for j in unvisited_indices:
                selection.append(
                    (pheromone_matrix[i][j] ** self.a) * ((1 / max(dist_matrix[i][j], 10**-5)) ** self.b)
                )
            selected_index = AntColonyOptimization.select_index(selection)
            visited_index.append(unvisited_indices.pop(selected_index))
        visited_index.append(visited_index[0])
        return visited_index

    def update_pheromone_matrix(self, pheromone_matrix: 'list[list[float]]', tmp_indx: 'list[list[int]]', tmp_leng: 'list[float]') -> None:
        """Обновляет матрицу феромонов."""

        l = len(pheromone_matrix)
        for i in range(l):
            for j in range(i, l):
                pheromone_matrix[i][j] *= 1 - self.p
                pheromone_matrix[j][i] *= 1 - self.p
        for i in range(self.num_ants):
            delta = self.q / tmp_leng[i]
            indx = tmp_indx[i]
            for j in range(l):
                pheromone_matrix[indx[j]][indx[j + 1]] += delta
                pheromone_matrix[indx[j + 1]][indx[j]] += delta

    def run(self, points: 'list[tuple[int]]') -> Path:
        """Запускает алгоритм для заданных точек."""

        l = len(points)
        dist_matrix = AntColonyOptimization.distance_matrix(points)
        pheromone_matrix = [[1 for _ in range(l)] for _ in range(l)]
        res_indx = []
        res_leng = inf
        for _ in range(self.num_histories):
            tmp_indx = []
            tmp_leng = []
            for _ in range(self.num_ants):
                indx = self.create_index(dist_matrix, pheromone_matrix)
                tmp_indx.append(indx)
                tmp_leng.append(AntColonyOptimization.path_distance(dist_matrix, indx))
            self.update_pheromone_matrix(pheromone_matrix, tmp_indx, tmp_leng)
            best_leng = min(tmp_leng)
            if best_leng < res_leng:
                res_leng = best_leng
                res_indx = tmp_indx[tmp_leng.index(best_leng)]
                self.temp_res.append(res_indx)
        return Path(points=res_indx, length=res_leng)


if __name__ == "__main__":
    ACO = AntColonyOptimization(num_ants=100, num_histories=20, a=1.5, b=1.2, p=0.6, q=10)    
    
    n = 100
    points = [(randint(0,500), randint(0, 500)) for i in range(n)]
    path = ACO.run(points)
    print(path.points)
    for res in ACO.temp_res:
        print(res)
        
    import matplotlib.pyplot as plt
    from matplotlib import animation

    fig = plt.figure()

    def init():
        X = [p[0] for p in points]
        Y = [p[1] for p in points]
        plt.plot(X, Y, 'r-')
        # print(X)
        
    
    def animate(i):
        plt.clf()
        if i < len(ACO.temp_res):
            data = ACO.temp_res[i]
            param = 'r-'
        else:
            data = ACO.temp_res[-1]
            param = 'b-'
        X = [points[ind][0] for ind in data]
        Y = [points[ind][1] for ind in data]
        
        plt.plot(X, Y, param)
        print(X)
    anim = animation.FuncAnimation(fig, animate, init_func=init, frames=len(ACO.temp_res) + 5, repeat = False, interval=500)
    anim.save('ant.gif')

![SegmentLocal](ant.gif "segment")